In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath = os.path.sep.join([r'__PATH__','deploy.prototxt'])
weightsPath = os.path.sep.join([r'__PATH__','res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
net = cv2.dnn.readNet(prototxtPath,weightsPath)

In [4]:
model = load_model(r'__PATH TO MODEL__\mask_detector.model')

In [5]:
image = cv2.imread(r'__PATH__\test images\example_04.jpg')

In [6]:
image

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[229, 220, 211],
        [237, 231, 224],
        [236, 230, 223],
        ...,
        [129, 129, 129],
        [255, 255, 255],
        [255, 255, 255]],

       [[233, 225, 218],
        [236, 230, 223],
        [236, 230, 223],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[236, 230, 223],
        [235, 229, 222],
        [237, 229, 222],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [254, 254, 254]]

In [7]:
(h,w) = image.shape[:2]

In [8]:
(h,w)

(320, 400)

In [9]:
blob = cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [10]:
blob

array([[[[ 151.,  151.,  151., ...,  151.,  151.,  151.],
         [ 151.,  151.,  151., ...,  151.,  151.,  151.],
         [ 151.,  151.,  151., ...,  151.,  151.,  151.],
         ...,
         [ 125.,  132.,  132., ...,  -49.,   91.,  151.],
         [ 129.,  132.,  131., ...,  130.,  144.,  151.],
         [ 132.,  132.,  134., ...,  151.,  151.,  150.]],

        [[  78.,   78.,   78., ...,   78.,   78.,   78.],
         [  78.,   78.,   78., ...,   78.,   78.,   78.],
         [  78.,   78.,   78., ...,   78.,   78.,   78.],
         ...,
         [  44.,   53.,   53., ..., -122.,   18.,   78.],
         [  48.,   53.,   52., ...,   57.,   71.,   78.],
         [  52.,   52.,   52., ...,   78.,   78.,   77.]],

        [[ 132.,  132.,  132., ...,  132.,  132.,  132.],
         [ 132.,  132.,  132., ...,  132.,  132.,  132.],
         [ 132.,  132.,  132., ...,  132.,  132.,  132.],
         ...,
         [  89.,  101.,  100., ...,  -68.,   72.,  132.],
         [  95.,  100.,   

In [11]:
blob.shape

(1, 3, 300, 300)

In [12]:
net.setInput(blob)
detections = net.forward()

In [13]:
detections

array([[[[0.        , 1.        , 0.9990723 , ..., 0.2609468 ,
          0.86956596, 0.5144971 ],
         [0.        , 1.        , 0.99022526, ..., 0.1156572 ,
          0.37667134, 0.371835  ],
         [0.        , 1.        , 0.13693598, ..., 0.16242325,
          0.3751283 , 0.2838493 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [21]:
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    
    
    if confidence > 0.5:
        box = detections[0,0,i,3:7] * np.array([w,h,w,h]) # Getting the x,y coordinates for creating the box, element wise multiplication
        (startX,startY,endX,endY) = box.astype('int')
        
        (startX,startY) = (max(0,startX),max(0,startY))
        (endX,endY) = (min(w-1,endX), min(h-1,endY))
        
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face = cv2.resize(face,(224,224))
        face = img_to_array(face)
        face = preprocess_input(face)
        
        face = np.expand_dims(face,axis=0)
       
        
        (mask,withoutMask)=model.predict(face)[0]
        
        label='Mask' if mask > withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("Result",image)
cv2.waitKey(0)
cv2.destroyAllWindows()